In [ ]:
import io
import pickle

from PIL import Image, ImageOps
import numpy as np
from matplotlib import pyplot as plt
import torch

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.multiclass import OneVsOneClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import accuracy_score, f1_score, adjusted_rand_score, confusion_matrix, ConfusionMatrixDisplay

from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from torchvision import transforms
from torchvision.models import VGG16_Weights

import pandas as pd
from IPython.display import display

# Initialing compute device (use GPU if available).
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'[torch] using {device}')

plt.ion()
plt.show()

### Dataset bootstrap

In [ ]:

# Load the dataset
raw_dataset = np.load("dataset_food101tiny.zip", allow_pickle=True)

dataset = {
    "train": {
        "data": [],
        "names": [],
        "labels": [],
        "unique_labels": [],
    },
    "valid": {
        "data": [],
        "names": [],
        "labels": [],
        "unique_labels": [],
    },
}

images_shape = (224, 224)

# For each image we have the path from which we extract the name and the label of the image
for dsKey in raw_dataset.keys():
    splittedKey = dsKey.split("/")

    img_type = splittedKey[2]
    img_label = splittedKey[3]
    img_name = splittedKey[4]

    img = Image.open(io.BytesIO(raw_dataset[dsKey]))
    img = ImageOps.fit(img, images_shape, Image.Resampling.LANCZOS).convert("RGB")

    img_array = np.asarray(img)

    dataset[img_type]["data"].append(img_array)
    dataset[img_type]["names"].append(img_name)
    dataset[img_type]["labels"].append(img_label)

for img_type in dataset.keys():
    dataset[img_type]["data"] = np.asarray(dataset[img_type]["data"])
    dataset[img_type]["names"] = np.asarray(dataset[img_type]["names"])

    dataset[img_type]["unique_labels"], dataset[img_type]["labels"] = np.unique(
        np.asarray(dataset[img_type]["labels"]), return_inverse=True
    )

### Feature extraction using VGG

Normalization mean and standard deviation are [here](https://pytorch.org/hub/pytorch_vision_vgg)

In [ ]:
normalization_std = [0.229, 0.224, 0.225]
normalization_mean = [0.485, 0.456, 0.406]

loader = transforms.Compose(
    [
        # transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=normalization_mean, std=normalization_std),
    ]
)

vgg_out = {"train": [], "valid": []}

# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16(weights=models.VGG16_Weights.DEFAULT).features.to(device)

for img_type in dataset.keys():
    vgg_out[img_type] = []

    for image_idx in range(dataset[img_type]["data"].shape[0]):
        loaded_image = (
            loader(dataset[img_type]["data"][image_idx, :]).unsqueeze(0).to(device)
        )

        with torch.no_grad():
            res = model(loaded_image)
        features = res.data.detach().cpu().numpy().flatten()
        print(f"Extracting feature: {image_idx}/{dataset[img_type]['data'].shape[0]}")

        vgg_out[img_type].append(features)

    vgg_out[img_type] = np.asarray(vgg_out[img_type])
    print(vgg_out[img_type].shape)

pickle.dump(vgg_out, open(".pkl/vgg_out.pkl", "wb"))

### Dimensionality reduction using PCA, LDA and t-SNE

In [ ]:
# Each method is tested with a set of numer of components
n_components_to_test = {
    "PCA": [3, 10, 50, 100, 200, 500, 1200],
    "LDA": [3, 5, 7, 9],
    "TSNE": [2, 3],
}

In [ ]:
# preload env
vgg_out = pickle.load(open(".pkl/vgg_out.pkl", "rb"))

# Results to compare the methods the number of component changes
results_PCA = []
results_LDA = []

PCAs_instances = {}
LDAs_instances = {}
TSNEs_instances = {}

PCAs_results = {
    "train": {},
    "valid": {},
}

LDAs_results = {
    "train": {},
    "valid": {},
}

TSNEs_results = {
    "train": {},
    "valid": {},
}

for n_components in n_components_to_test["PCA"]:
    print(f'[PCA] Extracting features (# components:{n_components})')
    
    PCAs_instances[n_components] = []

    PCAs_results["train"][n_components] = []
    PCAs_results["valid"][n_components] = []

    PCA_instance = PCA(n_components=n_components)

    PCA_instance.fit(vgg_out["train"])

    PCAs_results["train"][n_components] = PCA_instance.transform(vgg_out["train"])
    PCAs_results["valid"][n_components] = PCA_instance.transform(vgg_out["valid"])

    PCAs_instances[n_components] = PCA_instance
    
    results_PCA.append(
        {
            "METHOD": "PCA",
            "# Components": n_components,
            "CHANNEL": "RGB",
            "Explained Variance Ratio": np.sum(
                PCA_instance.explained_variance_ratio_, axis=0
            ),
        }
    )

for n_components in n_components_to_test["LDA"]:
    print(f'[LDA] Extracting features (# components:{n_components})')
    
    LDAs_instances[n_components] = []

    LDAs_results["train"][n_components] = []
    LDAs_results["valid"][n_components] = []

    LDA_instance = LinearDiscriminantAnalysis(n_components=n_components)

    LDA_instance.fit(vgg_out["train"], dataset["train"]["labels"])

    LDAs_results["train"][n_components] = LDA_instance.transform(vgg_out["train"])
    LDAs_results["valid"][n_components] = LDA_instance.transform(vgg_out["valid"])

    LDAs_instances[n_components] = LDA_instance

    results_LDA.append(
        {
            "METHOD": "LDA",
            "# Components": n_components,
            "CHANNEL": "RGB",
            "Explained Variance Ratio": np.sum(
                LDA_instance.explained_variance_ratio_, axis=0
            ),
        }
    )

for n_components in n_components_to_test["TSNE"]:
    print(f'[t-SNE] Extracting features (# components:{n_components})')

    TSNEs_instances[n_components] = []

    TSNEs_results["train"][n_components] = []

    TSNE_instance_train = TSNE(n_components=n_components, verbose=1, n_iter=3000)
    TSNE_instance_valid = TSNE(n_components=n_components, verbose=1, n_iter=3000)

    TSNEs_results["train"][n_components] = TSNE_instance_train.fit_transform(
        LDAs_results["train"][7]
    )

    TSNEs_results["valid"][n_components] = TSNE_instance_valid.fit_transform(
        LDAs_results["valid"][7]
    )

    TSNEs_instances[n_components] = [TSNE_instance_train, TSNE_instance_valid]


# Pandas tables
df_results_PCA = pd.DataFrame(results_PCA)
df_results_LDA = pd.DataFrame(results_LDA)

def highlight_cells(val):
    color = ""
    if val > 0.80:
        color = "background-color: lightgreen; color: black; font-weight: bold"
    elif val < 0.80:
        color = "background-color: lightcoral; color: black; font-weight: bold"
    return color

# Apply the style
df_results_PCA_styled = (
    df_results_PCA.style.map(highlight_cells, subset=["Explained Variance Ratio"])
    .set_caption("PCA Results")
    .set_properties(**{"text-align": "center"})
)

df_results_LDA_styled = (
    df_results_LDA.style.map(highlight_cells, subset=["Explained Variance Ratio"])
    .set_caption("LDA Results")
    .set_properties(**{"text-align": "center"})
)


pickle.dump(PCAs_results, open(".pkl/vgg_pca_out.pkl", "wb"))
pickle.dump(LDAs_results, open(".pkl/vgg_lda_out.pkl", "wb"))
pickle.dump(TSNEs_results, open(".pkl/vgg_tsne_out.pkl", "wb"))

display(df_results_PCA_styled)
display(df_results_LDA_styled)

### 2D/3D Data visualization

In [ ]:
vgg_out = pickle.load(open(".pkl/vgg_out.pkl", "rb"))
 
PCAs_results = pickle.load(open(".pkl/vgg_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/vgg_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/vgg_tsne_out.pkl", "rb"))

tSNE_fig_2D = plt.figure()
tSNE_3D = tSNE_fig_2D.add_subplot()

for i in range(len(dataset["train"]["unique_labels"])):
    classIdxs = dataset["train"]["labels"] == i

    tsne_features = TSNEs_results["train"][2][classIdxs, :]

    tSNE_3D.set_label(dataset["train"]["unique_labels"][i])
    tSNE_3D.scatter(
        tsne_features[:, 0],
        tsne_features[:, 1],
        marker=".",
        label=dataset["train"]["unique_labels"][i],
    )

plt.legend(loc="upper left")

# 3D plot
tSNE_fig_3D = plt.figure()
tSNE_3D = tSNE_fig_3D.add_subplot(projection="3d")

for i in range(len(dataset["train"]["unique_labels"])):
    classIdxs = dataset["train"]["labels"] == i

    tsne_features = TSNEs_results["train"][3][classIdxs, :]

    tSNE_3D.scatter(
        tsne_features[:, 0],
        tsne_features[:, 1],
        tsne_features[:, 2],
        marker=".",
        label=dataset["train"]["unique_labels"][i],
    )

plt.legend(loc="upper left")

### Classification - KNN

In [ ]:
vgg_out = pickle.load(open(".pkl/vgg_out.pkl", "rb"))
 
PCAs_results = pickle.load(open(".pkl/vgg_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/vgg_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/vgg_tsne_out.pkl", "rb"))

# Number of neighbors to test
k_to_test = {
    "VGG": [3, 5, 9, 15, 21, 55, 111, 251],
    "PCA": [3, 5, 9, 15, 21, 55, 111, 251],
    "LDA": [3, 5, 9, 15, 21, 55, 111, 251],
}

KNN_VGG_stats = []
KNN_PCA_stats = []
KNN_LDA_stats = []

for k_idx, k in enumerate(k_to_test["VGG"]):

    KNN_VGG_stats.insert(k_idx, [k])

    knn = OneVsOneClassifier(KNeighborsClassifier(k))

    knn.fit(vgg_out["train"], dataset["train"]["labels"])
    preds = knn.predict(vgg_out["valid"])

    accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
    precision = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

    KNN_VGG_stats[k_idx].append((accuracy, precision))

    # ConfusionMatrixDisplay(confusion_matrix(dataset['valid']['labels'], preds),display_labels=dataset['valid']['unique_labels']).plot()

KNN_VGG_df = pd.DataFrame(KNN_VGG_stats, columns=["k\\VGG", ""])
display(KNN_VGG_df)

for k_idx, k in enumerate(k_to_test["PCA"]):

    KNN_PCA_stats.insert(k_idx, [k])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        knn = OneVsOneClassifier(KNeighborsClassifier(k))

        knn.fit(PCAs_results["train"][n_components], dataset["train"]["labels"])
        preds = knn.predict(PCAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        precision = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        KNN_PCA_stats[k_idx].insert(n_components_idx + 1, (accuracy, precision))

        # ConfusionMatrixDisplay(confusion_matrix(dataset['valid']['labels'], preds),display_labels=dataset['valid']['unique_labels']).plot()

KNN_PCA_df = pd.DataFrame(
    KNN_PCA_stats, columns=["k\\PCA components"] + n_components_to_test["PCA"]
)
display(KNN_PCA_df)

for k_idx, k in enumerate(k_to_test["LDA"]):

    KNN_LDA_stats.insert(k_idx, [k])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        knn = OneVsOneClassifier(KNeighborsClassifier(k))

        knn.fit(LDAs_results["train"][n_components], dataset["train"]["labels"])
        preds = knn.predict(LDAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        precision = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        KNN_LDA_stats[k_idx].insert(n_components_idx + 1, (accuracy, precision))
        # ConfusionMatrixDisplay(confusion_matrix(dataset['valid']['labels'], preds),display_labels=dataset['valid']['unique_labels']).plot()


KNN_LDA_df = pd.DataFrame(
    KNN_LDA_stats, columns=["k\\LDA components"] + n_components_to_test["LDA"]
)
display(KNN_LDA_df)


### Classification - SVM

In [ ]:
vgg_out = pickle.load(open(".pkl/vgg_out.pkl", "rb"))
 
PCAs_results = pickle.load(open(".pkl/vgg_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/vgg_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/vgg_tsne_out.pkl", "rb"))

kernels_to_test = {
    "VGG": ["linear", "poly", "sigmoid"], 
    "PCA": ["linear", "poly", "sigmoid"], 
    "LDA": ["linear", "poly", "sigmoid"], 
}

SVM_VGG_stats = []
SVM_PCA_stats = []
SVM_LDA_stats = []

for kernel_idx, kernel in enumerate(kernels_to_test["PCA"]):

    SVM_PCA_stats.insert(kernel_idx, [kernel])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        svm = OneVsOneClassifier(SVC(kernel=kernel))

        svm.fit(PCAs_results["train"][n_components], dataset["train"]["labels"])

        preds = svm.predict(PCAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="weighted"), 3)

        SVM_PCA_stats[kernel_idx].insert(n_components_idx + 1, (accuracy, f_one_score))

SVM_PCA_df = pd.DataFrame(
    SVM_PCA_stats, columns=["kernel\\PCA components"] + n_components_to_test["PCA"]
)
display(SVM_PCA_df)

for kernel_idx, kernel in enumerate(kernels_to_test["LDA"]):

    SVM_LDA_stats.insert(kernel_idx, [kernel])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        svm = OneVsOneClassifier(SVC(kernel=kernel))

        svm.fit(LDAs_results["train"][n_components], dataset["train"]["labels"])

        preds = svm.predict(LDAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="weighted"), 3)

        SVM_LDA_stats[kernel_idx].insert(n_components_idx + 1, (accuracy, f_one_score))

SVM_LDA_df = pd.DataFrame(
    SVM_LDA_stats, columns=["kernel\\LDA components"] + n_components_to_test["LDA"]
)
display(SVM_LDA_df)


for kernel_idx, kernel in enumerate(kernels_to_test["VGG"]):

    SVM_VGG_stats.insert(kernel_idx, [kernel])

    svm = OneVsOneClassifier(SVC(kernel=kernel))

    svm.fit(vgg_out["train"], dataset["train"]["labels"])

    preds = svm.predict(vgg_out["valid"])

    accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
    f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="weighted"), 3)

    SVM_VGG_stats[kernel_idx].append((accuracy, f_one_score))

SVM_VGG_df = pd.DataFrame(
    SVM_VGG_stats, columns=["kernel\\VGG components", ""]
)
display(SVM_VGG_df)


### Classification - SGD

In [ ]:
vgg_out = pickle.load(open(".pkl/vgg_out.pkl", "rb"))
 
PCAs_results = pickle.load(open(".pkl/vgg_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/vgg_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/vgg_tsne_out.pkl", "rb"))

losses_to_test = {
    "VGG": ["modified_huber", "log_loss", "hinge"],
    "PCA": ["modified_huber", "log_loss", "hinge"],
    "LDA": ["modified_huber", "log_loss", "hinge"]
}

SGD_VGG_stats = []
SGD_PCA_stats = []
SGD_LDA_stats = []

for loss_idx, loss in enumerate(losses_to_test["VGG"]):

    SGD_VGG_stats.insert(loss_idx, [loss])

    sgd = SGDClassifier(loss=loss, max_iter=10000)

    sgd.fit(vgg_out["train"], dataset["train"]["labels"])

    preds = sgd.predict(vgg_out["valid"])

    accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
    f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="weighted"), 3)

    SGD_VGG_stats[loss_idx].append((accuracy, f_one_score))

    if loss == "modified_huber":
        ConfusionMatrixDisplay(confusion_matrix(dataset['valid']['labels'], preds),display_labels=dataset['valid']['unique_labels']).plot() 
        plt.xticks(rotation=90)

SGD_VGG_df = pd.DataFrame(
    SGD_VGG_stats, columns=["loss\\VGG", ""]
)
display(SGD_VGG_df)

for loss_idx, loss in enumerate(losses_to_test["PCA"]):

    SGD_PCA_stats.insert(loss_idx, [loss])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        sgd = SGDClassifier(loss=loss, max_iter=10000)

        sgd.fit(PCAs_results["train"][n_components], dataset["train"]["labels"])

        preds = sgd.predict(PCAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        SGD_PCA_stats[loss_idx].insert(
            n_components_idx + 1, (accuracy, f_one_score)
        )

SGD_PCA_df = pd.DataFrame(
    SGD_PCA_stats, columns=["loss\\PCA"] + n_components_to_test["PCA"]
)
display(SGD_PCA_df)

for loss_idx, loss in enumerate(losses_to_test["LDA"]):

    SGD_LDA_stats.insert(loss_idx, [loss])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        sgd = SGDClassifier(loss=loss, max_iter=10000)

        sgd.fit(LDAs_results["train"][n_components], dataset["train"]["labels"])

        preds = sgd.predict(LDAs_results["valid"][n_components])

        accuracy = round(accuracy_score(dataset["valid"]["labels"], preds), 3)
        f_one_score = round(f1_score(dataset["valid"]["labels"], preds, average="macro"), 3)

        SGD_LDA_stats[loss_idx].insert(
            n_components_idx + 1, (accuracy, f_one_score)
        )

SGD_LDA_df = pd.DataFrame(
    SGD_LDA_stats, columns=["loss\\LDA"] + n_components_to_test["LDA"]
)
display(SGD_LDA_df)

### Classification - K-Means

In [ ]:
vgg_out = pickle.load(open(".pkl/vgg_out.pkl", "rb"))
 
PCAs_results = pickle.load(open(".pkl/vgg_pca_out.pkl", "rb"))
LDAs_results = pickle.load(open(".pkl/vgg_lda_out.pkl", "rb"))
TSNEs_results = pickle.load(open(".pkl/vgg_tsne_out.pkl", "rb"))

algos_to_test = {
    "VGG": ["lloyd", "elkan"],
    "PCA": ["lloyd", "elkan"],
    "LDA": ["lloyd", "elkan"]
}

KMEANS_VGG_stats = []
KMEANS_PCA_stats = []
KMEANS_LDA_stats = []

for algo_idx, algorithm in enumerate(algos_to_test["VGG"]):

    KMEANS_VGG_stats.insert(algo_idx, [algorithm])

    kmeans = KMeans(n_clusters=10, n_init='auto')

    kmeans.fit(vgg_out["train"])

    preds = kmeans.predict(vgg_out["valid"])

    score = round(adjusted_rand_score(dataset["valid"]["labels"], preds), 3)

    KMEANS_VGG_stats[algo_idx].append(score)

KMEANS_VGG_df = pd.DataFrame(
    KMEANS_VGG_stats, columns=["algorithm\\VGG", ""]
)
display(KMEANS_VGG_df)

for algo_idx, algorithm in enumerate(algos_to_test["PCA"]):

    KMEANS_PCA_stats.insert(algo_idx, [algorithm])

    for n_components_idx, n_components in enumerate(n_components_to_test["PCA"]):
        kmeans = KMeans(algorithm=algorithm, n_init='auto')

        kmeans.fit(PCAs_results["train"][n_components])

        preds = kmeans.predict(PCAs_results["valid"][n_components])

        score = round(adjusted_rand_score(dataset["valid"]["labels"], preds), 3)

        KMEANS_PCA_stats[algo_idx].insert(n_components_idx + 1, score)

KMEANS_PCA_df = pd.DataFrame(
    KMEANS_PCA_stats, columns=["algorithm\\PCA"] + n_components_to_test["PCA"]
)
display(KMEANS_PCA_df)

for algo_idx, algorithm in enumerate(algos_to_test["LDA"]):

    KMEANS_LDA_stats.insert(algo_idx, [algorithm])

    for n_components_idx, n_components in enumerate(n_components_to_test["LDA"]):
        kmeans = KMeans(algorithm=algorithm, n_init='auto')

        kmeans.fit(LDAs_results["train"][n_components])

        preds = kmeans.predict(LDAs_results["valid"][n_components])

        score = round(adjusted_rand_score(dataset["valid"]["labels"], preds), 3)

        KMEANS_LDA_stats[algo_idx].insert(n_components_idx + 1, score)

KMEANS_LDA_df = pd.DataFrame(
    KMEANS_LDA_stats, columns=["algorithm\\LDA"] + n_components_to_test["LDA"]
)

display(KMEANS_LDA_df)